In [3]:
import tensorflow as tf
import nltk
from nltk.corpus import gutenberg

# word embedding
词向量模型，主要功能是用于表示

参考
1）cs224n 第二讲和第三讲
2）speech and language processing chapter 16 -- semantic with dense vectors

# 文本表征的方法
文本表征的目的是将文本转化成机器学习算法可以处理的数字形式，换句话说就是将文本使用数学语言进行表示。
- 词袋模型 one-hot\tf-idf
- 主题模型，LSA\PLSA\LDA等
- 词向量模型 word2vec fastText glove
- 动态词向量模型 elmo BERT GPT

词向量模型将一个词表示成一个先对要短（1000以内的长度）以及稠密的向量（大部分值不为0）
词袋模型简单，但是存在维数灾难和语义鸿沟的问题
主题模型训练计算复杂度高
早期的词向量的研究源于语言模型，比附NNLM和RNNLM, 词向量是副产物。
**语言模型，就是用来计算一个词序列的概率的模型。**


分布式假设：相同上下文语境的词具有相似含义，并且由此引出了word2vec和fastText等。这类词向量中，虽然其本质仍然是语言模型，但其目标是



In [5]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [6]:
data = gutenberg.words("austen-emma.txt")

In [8]:
word_id = 

['[', 'Emma']